In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # to disable GPU
import featureEng as fe
import processStocks as ps
import getStocks
import visualize
import predicts
import pandas as pd
import TS_LSTM as ts_lstm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [ ]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# User parameters

In [ ]:
ticker = 'amd'.upper()
n_in = 100 # number of historical days to input
n_out = 1 # days into future to predict
target = 'Close'

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

In [ ]:
# load dataset
# dataset = getStocks.get_single(ticker=ticker, save=True)
dataset = getStocks.load_single(ticker)
#dataset = getStocks.join_tgt_spt(target_ticker=ticker.upper(), number_spt=20)
dataset.reset_index(level=0, inplace=True)
dataset.rename(columns=lambda x: x.replace(' ',''),inplace=True)

# Prepare Data for LSTM

In [ ]:
# using default options
'''
ts_data=ts_lstm.ts_data.default_prep(n_in=n_in, n_out=n_out, 
                      ticker=ticker,target=target,
                      rawData=dataset)
'''

#
ts_data = ts_lstm.ts_data(n_in=n_in, n_out=n_out, 
                      ticker=ticker,target=target,
                      rawData=dataset)
ts_data.eng_features(derivate=False, weekdays=False)
ts_data.roll_data()
ts_data.tscv()

# Build LSTM Architecture

In [ ]:
ts_model = ts_lstm.build_model(ts_data, 
                               inlayer=int(ts_data.train_X.shape[-1])*2,
                               hiddenlayers=[5,3],
                               loss_function='mean_squared_error',
                               dropout=0.05,
                               activation='sigmoid')

# Fit model

In [ ]:
history = ts_model.fit(ts_data.train_X, ts_data.train_y, 
                    epochs=300, 
                    batch_size=1024, 
                    validation_data=(ts_data.test_X, ts_data.test_y), 
                    verbose=2, 
                    shuffle=False)

In [ ]:
visualize.plot_loss(history)

# Visualize Results

In [ ]:
yhat = predicts.predict_sequences_multiple(ts_model, ts_data.test_X, ts_data.n_in, ts_data.n_out)

In [ ]:
import matplotlib.pyplot as plt
x = range(len(yhat))
plt.plot(x, ts_data.test_y, label='actual', )
plt.plot(x, yhat, label='predict')
plt.legend()
plt.show()